In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [81]:
df = pd.read_csv(r'C:\Zeel\UChicago\Winter\Real Time Intelligence Systems\Project Phase 1\Data\historical_stock_data_5min_6months.csv', parse_dates=['timestamp'])

In [82]:
df.head()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2024-08-26 08:00:00+00:00,226.25,226.41,226.25,226.41,1965,219,226.365909
1,AMZN,2024-08-26 08:00:00+00:00,177.00,177.60,177.00,177.30,6356,150,177.363163
2,AVGO,2024-08-26 08:00:00+00:00,165.75,165.75,165.75,165.75,359,13,165.750000
3,BAC,2024-08-26 08:00:00+00:00,39.32,39.34,39.31,39.34,1742,32,39.320000
4,DIS,2024-08-26 08:00:00+00:00,90.25,90.39,90.25,90.39,651,44,90.294025


In [83]:
print("Raw data shape:", df.shape)
print("\nFirst 3 rows:")
print(df.head(3))

Raw data shape: (843695, 9)

First 3 rows:
  symbol                 timestamp    open    high     low   close  volume  \
0   AAPL 2024-08-26 08:00:00+00:00  226.25  226.41  226.25  226.41    1965   
1   AMZN 2024-08-26 08:00:00+00:00  177.00  177.60  177.00  177.30    6356   
2   AVGO 2024-08-26 08:00:00+00:00  165.75  165.75  165.75  165.75     359   

   trade_count        vwap  
0          219  226.365909  
1          150  177.363163  
2           13  165.750000  


In [84]:
# Check data integrity
print("Data Validation:")
print(f"Total Rows: {len(df)}")
print(f"Missing Values: {df.isnull().sum().sum()}")
print(f"Duplicates: {df.duplicated(subset=['symbol', 'timestamp']).sum()}")

# Check time range for a sample symbol (e.g., AAPL)
aapl_times = df[df['symbol'] == 'AAPL']['timestamp'].agg(['min', 'max'])
print("\nAAPL Time Range:")
print(f"Start: {aapl_times['min']}\nEnd: {aapl_times['max']}")

Data Validation:
Total Rows: 843695
Missing Values: 0
Duplicates: 0

AAPL Time Range:
Start: 2024-08-26 08:00:00+00:00
End: 2025-02-19 23:50:00+00:00


In [88]:
df.describe()

,open,high,low,close,volume,trade_count,vwap
count,843695.000000,843695.000000,843695.000000,843695.000000,8.436950e+05,843695.000000,843695.000000
mean,253.247167,253.433196,253.055001,253.246885,1.532779e+05,1486.122346,253.245842
std,212.964985,213.119855,212.806106,212.965551,6.934574e+05,4026.523147,212.964672
min,18.525000,18.560000,18.510000,18.525000,1.000000e+02,1.000000,18.534677
25%,89.820000,89.880000,89.740000,89.810000,3.436500e+03,57.000000,89.810000
50%,190.010000,190.169200,189.830000,190.010000,2.575100e+04,601.000000,190.002057
75%,369.460000,369.830000,369.105000,369.470000,9.383200e+04,1328.000000,369.485745
max,1078.181000,1078.235000,1078.000000,1078.000000,1.307828e+08,356453.000000,1078.000000


In [89]:
df.head()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2024-08-26 08:00:00+00:00,226.25,226.41,226.25,226.41,1965,219,226.365909
1,AMZN,2024-08-26 08:00:00+00:00,177.00,177.60,177.00,177.30,6356,150,177.363163
2,AVGO,2024-08-26 08:00:00+00:00,165.75,165.75,165.75,165.75,359,13,165.750000
3,BAC,2024-08-26 08:00:00+00:00,39.32,39.34,39.31,39.34,1742,32,39.320000
4,DIS,2024-08-26 08:00:00+00:00,90.25,90.39,90.25,90.39,651,44,90.294025


In [92]:
df_sorted = df.sort_values(['symbol', 'timestamp']).reset_index(drop=True)

In [95]:
df_sorted.head()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2024-08-26 08:00:00+00:00,226.25,226.41,226.25,226.41,1965,219,226.365909
1,AAPL,2024-08-26 08:05:00+00:00,226.32,226.81,226.32,226.80,5482,235,226.670652
2,AAPL,2024-08-26 08:10:00+00:00,226.72,226.72,226.66,226.66,1918,53,226.712568
3,AAPL,2024-08-26 08:15:00+00:00,226.71,226.84,226.71,226.82,1427,50,226.792500
4,AAPL,2024-08-26 08:20:00+00:00,226.84,226.84,226.84,226.84,369,16,226.840000


In [96]:
import pandas as pd
import numpy as np

# Ensure your DataFrame is sorted and has a 'close' column
# 1. Calculate the EMAs
df_sorted['ema_12'] = df_sorted['close'].ewm(span=12, adjust=False).mean()
df_sorted['ema_26'] = df_sorted['close'].ewm(span=26, adjust=False).mean()

# 2. MACD line = ema_12 - ema_26
df_sorted['macd'] = df_sorted['ema_12'] - df_sorted['ema_26']

# 3. Signal line (9-day EMA of MACD)
df_sorted['signal_line'] = df_sorted['macd'].ewm(span=9, adjust=False).mean()

# 4. MACD histogram (optional, often used for visualization)
df_sorted['macd_hist'] = df_sorted['macd'] - df_sorted['signal_line']

# 5. Trading signal (for example):
#    +1 when MACD > Signal (bullish)
#    -1 when MACD < Signal (bearish)
df_sorted['macd_signal'] = 0
df_sorted.loc[df_sorted['macd'] > df_sorted['signal_line'], 'macd_signal'] = 1
df_sorted.loc[df_sorted['macd'] < df_sorted['signal_line'], 'macd_signal'] = -1

